In [1]:
import pulp
import pandas as pd
import numpy as np

def load_data():
    attachment1_path = r"C:/Users/81148/Desktop/C题/附件1.xlsx"
    attachment2_path = r"C:/Users/81148/Desktop/C题/附件2.xlsx"

    fields = pd.read_excel(attachment1_path, sheet_name=0)
    fields = fields.rename(columns={'地块名称': 'name', '地块类型': 'type', '地块面积/亩': 'area'})
    fields = fields[['name','type','area']]

    crops = pd.read_excel(attachment1_path, sheet_name=2)
    crops = crops.rename(columns={'作物编号': 'id', '作物名称': 'name', '种植耕地': 'suitable_fields'})
    crops = crops.dropna(subset=['id'])
    crops = crops[crops['id'].apply(lambda x: str(x).isdigit())]
    crops['id'] = crops['id'].astype(int)
    
    land_types = ['平旱地', '梯田', '山坡地', '水浇地', '普通大棚', '智慧大棚']
    quarters = ['第一季', '第二季']

    # 为每种土地和季度创建对应的列
    for land in land_types:
        for quarter in quarters:
            column_name = f'{land}({quarter})'
            crops[column_name] = 0  # 初始化所有列为0

    # 遍历每一行，根据'suitable_fields'的值更新相应的列
    for index, row in crops.iterrows():
        suitable_fields = row['suitable_fields'].split('，')  # 使用中文逗号分割多个耕地类型
        for field in suitable_fields:
            for land in land_types:
            # 处理 "第一季" 的情况
                if f'{land}(第一季)' in field:
                    crops.at[index, f'{land}(第一季)'] = 1
            # 处理 "第二季" 的情况
                elif f'{land}(第二季)' in field:
                    crops.at[index, f'{land}(第二季)'] = 1
            # 处理没有具体季度限制的情况
                elif land in field and '(第一季)' not in field and '(第二季)' not in field:
                    crops.at[index, f'{land}(第一季)'] = 1
                    crops.at[index, f'{land}(第二季)'] = 1
    
    # 删除原始的'种植耕地'列
    crops = crops.drop('suitable_fields', axis=1)

    planting_2023 = pd.read_excel(attachment2_path, sheet_name=0)
    planting_2023 = planting_2023.rename(columns={'种植地块': 'field', '作物编号': 'crop_id', '作物名称': 'crop_name', 
                                                  '作物类型': 'crop_type', '种植面积/亩': 'area', '种植季次': 'season'})

    stats_2023 = pd.read_excel(attachment2_path, sheet_name=1)
    stats_2023 = stats_2023.rename(columns={'序号': 'index', '作物编号': 'crop_id', '作物名称': 'crop_name', 
                                            '地块类型': 'field_type', '种植季次': 'season', '亩产量/斤': 'yield_per_mu',
                                            '种植成本/(元/亩)': 'cost_per_mu', '销售单价/(元/斤)': 'price_range'})
    stats_2023 = stats_2023.dropna(subset=['crop_id'])
    stats_2023 = stats_2023[stats_2023['crop_id'].apply(lambda x: str(x).isdigit())]
    stats_2023['crop_id'] = stats_2023['crop_id'].astype(int)
    
    stats_2023[['min_price', 'max_price']] = stats_2023['price_range'].str.split('-', expand=True).astype(float)
    stats_2023['avg_price'] = (stats_2023['min_price'] + stats_2023['max_price']) / 2
    stats_2023 = stats_2023.drop('price_range', axis=1)
    
    expected_sales = {}
    for _, row in planting_2023.iterrows():
        crop = row['crop_id']
        area = row['area']
        field = row['field']
        field_type = fields.loc[fields['name'] == field]['type'].values[0]
        yield_per_mu_1 = stats_2023.loc[(stats_2023['crop_id'] == crop)&(stats_2023['field_type'] == field_type), 'yield_per_mu'].values[0]
        #yield_per_mu_1 = stats_2023.query(f"crop_id == @crop and field_type == @field_type")['yield_per_mu'].values[0]
        if crop in expected_sales:
            expected_sales[crop] += area * yield_per_mu_1
        else:
            expected_sales[crop] = area * yield_per_mu_1
     


    print("Crops:")
    print(crops)
    print("\nStats 2023:")
    print(stats_2023)

    return fields, crops, planting_2023, stats_2023, expected_sales


In [3]:
%%capture
fields, crops, planting_2023, stats_2023, expected_sales = load_data();

In [75]:
print(expected_sales)

{6: 170840.0, 7: 132750.0, 1: 57000.0, 4: 33040.0, 8: 71400.0, 2: 21850.0, 3: 22400.0, 5: 9875.0, 9: 30000.0, 10: 12500.0, 14: 14000.0, 15: 10000.0, 11: 1500.0, 12: 35100.0, 13: 36000.0, 20: 30000.0, 36: 100000.0, 28: 35600.0, 35: 150000.0, 21: 36300.0, 22: 45600.0, 17: 36480.0, 18: 26880.0, 37: 36000.0, 16: 21000.0, 38: 9000.0, 24: 2700.0, 25: 3600.0, 26: 4050.0, 39: 7200.0, 27: 4500.0, 19: 6480.0, 40: 18000.0, 41: 4200.0, 29: 13500.0, 30: 3000.0, 31: 1200.0, 32: 3600.0, 33: 1800.0, 34: 1980.0, 23: 990.0}


In [5]:
def calculate_cost():
    attachment_path = r"C:/Users/81148/Desktop/C题/附件2.xlsx"
    costs = pd.read_excel(attachment_path, sheet_name=1)
    costs = costs[['作物编号','种植季次','种植成本/(元/亩)']]
    costs = costs.rename(columns={'作物编号': 'crop_id','种植季次': 'season','种植成本/(元/亩)': 'cost_per_mu',})
            
    costs = costs.drop('season', axis=1)
    return costs

In [7]:
%%capture
costs = calculate_cost();

In [11]:
costs.head(10)

,crop_id,cost_per_mu
0,1,200.0
1,2,200.0
2,3,175.0
3,4,175.0
4,5,175.0
5,6,225.0
6,7,250.0
7,8,180.0
8,9,200.0
9,10,180.0


In [9]:
def create_model(crops, fields, planting_2023, stats_2023, years, seasons, expected_sales,costs, case):
    model = pulp.LpProblem("Crop_Planning", pulp.LpMaximize)
    
    print("Crops:", crops['id'].tolist())
    print("Stats crop_id:", stats_2023['crop_id'].tolist())
    
    # 创建决策变量
    # 作物id   
    x = pulp.LpVariable.dicts("plant", 
                ((int(c), str(f), int(y), str(s)) 
                 for c in crops['id'] 
                 for f in fields['name'] 
                 for y in years 
                 for s in seasons),
                lowBound=0, cat='Continuous')
    
    excess = pulp.LpVariable.dicts("excess",
                ((int(c), int(y)) for c in crops['id'] for y in years),
                lowBound=0, cat='Continuous')
    
    #二进制变量表示是否种植
    z = pulp.LpVariable.dicts("is_planted", 
                ((int(c), str(f), int(y), str(s)) 
                 for c in crops['id'] 
                 for f in fields['name'] 
                 for y in years 
                 for s in seasons),
                cat='Binary')



    
    
    # 目标函数
    if case == 1:
        model += pulp.lpSum(x[int(c),str(f),int(y),str(s)] * stats_2023[stats_2023['crop_id'] == c]['yield_per_mu'].iloc[0] * 
                            stats_2023[stats_2023['crop_id'] == c]['avg_price'].iloc[0] -
                            x[int(c),str(f),int(y),str(s)] * costs[costs['crop_id'] == c]['cost_per_mu'].iloc[0]
                            for c in crops['id'] for f in fields['name'] for y in years for s in seasons
                            if not stats_2023[stats_2023['crop_id'] == c].empty)
    elif case == 2:
        model += pulp.lpSum(x[int(c),str(f),int(y),str(s)] * stats_2023[stats_2023['crop_id'] == c]['yield_per_mu'].iloc[0] * 
                            stats_2023[stats_2023['crop_id'] == c]['avg_price'].iloc[0] -
                            x[int(c),str(f),int(y),str(s)] * costs[costs['crop_id'] == c]['cost_per_mu'].iloc[0] +
                            excess[int(c),int(y)] * stats_2023[stats_2023['crop_id'] == c]['avg_price'].iloc[0] * 0.5
                            for c in crops['id'] for f in fields['name'] for y in years for s in seasons
                            if not stats_2023[stats_2023['crop_id'] == c].empty)
    
    # 约束条件
    # 1. 土地面积约束
    for f in fields['name']:
        for y in years:
            for s in seasons:
                model += pulp.lpSum(x[int(c),str(f),int(y),str(s)] for c in crops['id']) <= fields[fields['name'] == f]['area'].iloc[0]
                                                                              
    # 2. 作物轮作约束                   
    for y in years[1:]:  # 从第二年开始
        for f in fields['name']:
            for c in crops['id']:
                model += x[int(c),str(f),int(y),'第一季'] + x[int(c),str(f),int(y),'第二季'] + \
                        x[int(c),str(f),int(y)-1,'第一季'] + x[int(c),str(f),int(y)-1,'第二季'] <= \
                        fields.loc[fields['name'] == f, 'area'].values[0]
     
    # 3. 豆类种植约束   
    bean_crops = crops[crops['name'].str.contains('豆类', na=False)]['id']
    for f in fields['name']:
        for y in range(0, len(years), 3):
            model += pulp.lpSum(x[int(c),str(f),int(y),str(s)] 
                           for s in seasons 
                           for c in bean_crops) >= \
                fields.loc[fields['name'] == f, 'area'].values[0]
        
    # 4. 产量约束
    for c in crops['id']:
        for y in years:
            total_production = pulp.lpSum(x[int(c),str(f),int(y),str(s)] * stats_2023[stats_2023['crop_id'] == c]['yield_per_mu'].iloc[0]
                                          for f in fields['name'] for s in seasons)
            pred_sales = expected_sales[c] 
            model += total_production - pred_sales <= excess[int(c),int(y)]
        
    # 5. 最小种植面积的约束
    for c in crops['id']:
        for f in fields['name']:
            for y in years:
                for s in seasons:
                    area_min = fields[fields['name'] == f]['area'].iloc[0] * 0.1
                    model += x[int(c), str(f), int(y), str(s)] >= area_min * z[int(c), str(f), int(y), str(s)]  # 确保种植时达到最小面积
                    model += x[int(c), str(f), int(y), str(s)] <= fields[fields['name'] == f]['area'].iloc[0] * z[int(c), str(f), int(y), str(s)]  # 确保未选择种植时面积为0
                    
    # 6. 作物种植约束:                
    for y in years:
        for s in seasons:
            for f in fields['name']:
                for c in crops['id']:
                    land_type = fields.loc[fields['name'] == f, 'type'].values[0]
                    field_name = f'{land_type}({s})'
                    if crops.loc[crops['id'] == c, field_name].iloc[0] == 0:
                        model += x[int(c),str(f),int(y),str(s)] == 0


    # 7. 季节性约束：
    for y in years:
        for s in seasons:
            for f in fields['name']:
                for c in crops['id']:
                    land_type = fields.loc[fields['name'] == f, 'type'].values[0]
                    if land_type in ['平旱地', '梯田', '山坡地']:  #都是单季作物
                        if s == '第二季':
                            for c in crops['id']:
                                model += x[int(c),str(f),int(y),str(s)] == 0
                    elif land_type == '水浇地':
                        if s == '第一季':
                            model += x[16,str(f),int(y),str(s)] == 0  # 水稻
                        else:
                            for c in crops['id']:
                                if c not in [35, 36, 37]:  # 大白菜、白萝卜和红萝卜
                                    model += x[int(c),str(f),int(y),str(s)] == 0
                    elif land_type == '普通大棚':
                        if s == '第二季':
                            for c in crops['id']:
                                if c not in [38, 39, 40, 41]:  # 食用菌的编号
                                    model += x[int(c),str(f),int(y),str(s)] == 0
                    elif land_type == '智慧大棚':
                        for crop in [35, 36, 37]:  # 大白菜、白萝卜和红萝卜不能种在智慧大棚
                            model += x[int(c),str(f),int(y),str(s)] == 0
    
    # 8. 分散度约束
    for y in years:
        for s in seasons:
            for f in fields['name']:
                model += pulp.lpSum(z[int(c), str(f), int(y), str(s)] for crop in crops['id']) <= 3  # 每季最多种3种作物

    return model            


In [ ]:
'''   
# 5. 最小种植面积约束:
    for y in years:
        for f in fields['name']:
            land_type = fields.loc[fields['name'] == f, 'type'].values[0]
            land_area = fields.loc[fields['name'] == f, 'area'].values[0]
            min_area = 0.1 * land_area  # 最小种植面积为地块面积的10%
            for s in seasons:
                for c in crops['id']:
                    model += x[int(c),str(f),int(y),str(s)] >= min_area 


# 5. 种植分散度约束
    max_fields = 5
    for c in crops['id']:
        for y in years:
            for s in seasons:
                model += pulp.lpSum(pulp.LpVariable(f"plant_{c}_{f}_{y}_{s}_binary", cat='Binary') 
                                    for f in fields['name']) <= max_fields
                for f in fields['name']:
                    model += x[int(c),str(f),int(y),str(s)] <= fields[fields['name'] == f]['area'].iloc[0] * pulp.LpVariable(f"plant_{c}_{f}_{y}_{s}_binary", cat='Binary')
    
    # 6. 最小种植面积约束
    min_area = 10
    for c in crops['id']:
        for f in fields['name']:
            for y in years:
                for s in seasons:
                    model += (x[int(c),str(f),int(y),str(s)] >= min_area) * pulp.LpVariable(f"plant_{c}_{f}_{y}_{s}_min_area", cat='Binary')
                    model += x[int(c),str(f),int(y),str(s)] <= fields[fields['name'] == f]['area'].iloc[0] * pulp.LpVariable(f"plant_{c}_{f}_{y}_{s}_min_area", cat='Binary')
'''    


In [11]:
def solve_model(model):
    solver = pulp.PULP_CBC_CMD(msg=False, timeLimit=600)
    model.solve(solver)
    return model

In [13]:
def process_results(model, crops, fields, years, seasons):
    results = []
    for v in model.variables():
        if v.varValue > 0 and v.name.startswith("plant_"):
            _, c, f, y, s = v.name.split('_')
            results.append({
                'crop': crops[crops['id'] == int(c.strip('()').split(',')[0])]['name'].iloc[0],
                'field': f,
                'year': int(y.strip(',')),
                'season': s,
                'area': v.varValue
            })
    return pd.DataFrame(results)

In [57]:
def main():
    fields, crops, planting_2023, stats_2023, expected_sales = load_data()
    costs = calculate_cost();
    years = range(2024, 2031)
    seasons = ['第一季', '第二季']
    
    for case in [1, 2]:
        model = create_model(crops, fields, planting_2023, stats_2023, years, seasons, expected_sales, costs, case)
        solved_model = solve_model(model)
        results = process_results(solved_model, crops, fields, years, seasons)
        
        filename = f'C:/Users/81148/Desktop/C题/result1_{case}.xlsx'
        results.to_excel(filename, index=False)
        print(f"Results for case {case} saved to {filename}")

if __name__ == "__main__":
    main()

Crops:
    id name  平旱地(第一季)  平旱地(第二季)  梯田(第一季)  梯田(第二季)  山坡地(第一季)  山坡地(第二季)  \
0    1   黄豆         1         1        1        1         1         1   
1    2   黑豆         1         1        1        1         1         1   
2    3   红豆         1         1        1        1         1         1   
3    4   绿豆         1         1        1        1         1         1   
4    5   爬豆         1         1        1        1         1         1   
5    6   小麦         1         1        1        1         1         1   
6    7   玉米         1         1        1        1         1         1   
7    8   谷子         1         1        1        1         1         1   
8    9   高粱         1         1        1        1         1         1   
9   10   黍子         1         1        1        1         1         1   
10  11   荞麦         1         1        1        1         1         1   
11  12   南瓜         1         1        1        1         1         1   
12  13   红薯         1         1        1    